In [126]:
import sys
import re
import time
import datetime
import pandas as pd
import numpy as np
import func
# inline plot
%matplotlib inline

In [2]:
all_trains = pd.read_csv("data/weather-trains_2015-06-08 16:57:272016-01-25 23:42:14.csv",sep='\t',error_bad_lines=False)
delayed = pd.read_csv("data/merged_delay_2016-03-27-02:02:47.csv",sep='\t',error_bad_lines=False)
del delayed['Unnamed: 0']
del all_trains['Unnamed: 0']

In [3]:
print all_trains.shape
print delayed.shape

(21021, 12)


In [4]:
del all_trains['time_pa']
all_trains.head()

,train_id,stop_pa,t_northbound,timestamp,temp,precipiation,visability,windspeed,humidity,cloudcover
0,102,1,0,2015-06-08 05:53:00,72.97875,0,10,6.68125,0.62875,0.09875
1,104,1,0,2015-06-08 06:23:00,72.97875,0,10,6.68125,0.62875,0.09875
2,206,1,0,2015-06-08 06:59:00,72.97875,0,10,6.68125,0.62875,0.09875
3,208,1,0,2015-06-08 07:20:00,72.97875,0,10,6.68125,0.62875,0.09875
4,210,1,0,2015-06-08 07:28:00,72.97875,0,10,6.68125,0.62875,0.09875


In [108]:
delayed.columns.values

array(['day_of_week', 'day_of_month', 'month', 'time_of_day',
       'topic_train', 't_bullet', 't_limited', 't_northbound', 'timestamp',
       'train_id_x', 'tweet_id', 'uid', 'is_delay', 'delay_minor',
       'delay_med', 'delay_major', 'delay_catastrophic', 'is_backlog',
       'is_canceled', 'is_passing', 'is_accident', 'is_medical',
       'is_mechanical', 'is_customer', 'is_event', 'id', 'temp',
       'precipiation', 'visability', 'windspeed', 'humidity', 'cloudcover'], dtype=object)

In [114]:
delayed.head(3)

,day_of_week,day_of_month,month,time_of_day,topic_train,t_bullet,t_limited,t_northbound,timestamp,train_id_x,...,is_mechanical,is_customer,is_event,id,temp,precipiation,visability,windspeed,humidity,cloudcover
0,0,25,1,error,'155',0,0,1,2015-06-08 05:53:00,155,...,0,0,0,691768068385718275,52.890417,0.000000,10.000000,3.979583,0.811667,0.1555
1,4,22,1,error,'151',0,0,1,2015-06-08 06:23:00,151,...,0,0,0,690667494906814464,59.498750,0.003975,8.580000,9.688750,0.820833,0.2325
2,2,20,1,error,'151',0,0,1,2015-06-08 06:59:00,151,...,0,0,0,689936168893329408,53.795000,0.000000,8.801667,3.730833,0.877917,0.1080


In [117]:
all_trains.head(3)

,train_id,stop_pa,t_northbound,timestamp,temp,precipiation,visability,windspeed,humidity,cloudcover
0,102,1,0,2015-06-08 05:53:00,72.97875,0,10,6.68125,0.62875,0.09875
1,104,1,0,2015-06-08 06:23:00,72.97875,0,10,6.68125,0.62875,0.09875
2,206,1,0,2015-06-08 06:59:00,72.97875,0,10,6.68125,0.62875,0.09875


In [ ]:
# merged = all_trains.merge(atw,left_on='time_pa',right_on='timestamp',how='outer')
# print merged.shape
# merged.head(5)

In [ ]:
# all_trains.merge(delayed,left_on=['train_id'])

In [7]:
all_trains['timestamp'] = pd.to_datetime(all_trains['timestamp'],format="%Y-%m-%d %H:%M")

In [38]:
delayed['timestamp'] = pd.to_datetime(all_trains['timestamp'],format="%Y-%m-%d %H:%M")

In [40]:
max_date = all_trains['timestamp'].max()
min_date = all_trains['timestamp'].min()
print max_date
print min_date

2016-01-24 23:38:00
2015-06-08 05:01:00


In [101]:
all_trains.shape

(21021, 10)

In [173]:
import calendar
def populate_all_trains(t_row):
    t=t_row['timestamp']
    next_mo = True if t.day >= calendar.monthrange(t.year,t.month)[1] else False
    year2 = t.year if next_mo is False & t.month != 12 else t.year+1
    month2 = t.month if next_mo is False else (t.month + 1 if (t.month+1)<=12 else 1)
    day2 = t.day+1 if next_mo is False else 1
    
    d = delayed[(delayed['timestamp'] > datetime.datetime(t.year,t.month,t.day)) 
           & (delayed['timestamp'] <= datetime.datetime(year2,month2,day2))
            & (delayed['train_id_x'] == t_row['train_id'])]

    if d.empty is False:
        return pd.Series([
            d.t_bullet.max(),
            d.t_limited.max(),
            d.uid.max(),
            d.is_delay.max(),
            d.delay_minor.max(),
            d.delay_med.max(),
            d.delay_major.max(),
            d.delay_catastrophic.max(),
            d.is_backlog.max(),
            d.is_canceled.max(),
            d.is_passing.max(),
            d.is_accident.max(),
            d.is_medical.max(),
            d.is_mechanical.max(),
            d.is_customer.max(),
            d.is_event.max()])
    else:
        return pd.Series([
            1 if str(t_row.train_id)[0] == 3 else 0,
            1 if str(t_row.train_id)[0] == 2 else 0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0])

In [174]:
test = all_trains.apply(lambda x:populate_all_trains(x),axis=1)

In [175]:
test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,258611670061820981248,1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,231646719505939324929,1,0,0,0,0,1,0,1,0,0,0,0,0


In [176]:
test.describe()

,0,1,4,5,6,7,8,9,10,11,12,13,14,15
count,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000,21021.000000
mean,0.008277,0.014890,0.015318,0.020931,0.006422,0.002949,0.009562,0.004805,0.003282,0.008040,0.002997,0.014937,0.007469,0.003568
std,0.090605,0.121115,0.122817,0.143158,0.079882,0.054230,0.097318,0.069151,0.057200,0.089305,0.054664,0.121305,0.086100,0.059626
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [180]:
test.columns = ['is_bullet','is_limited','tweet_id','is_delay','del_min','del_med','del_maj',
                'del_cat','is_backlog','is_canceled','is_passing','is_accident','is_medical',
                'is_mechanical','is_customer','is_event']

In [181]:
test.head()

,is_bullet,is_limited,tweet_id,is_delay,del_min,del_med,del_maj,del_cat,is_backlog,is_canceled,is_passing,is_accident,is_medical,is_mechanical,is_customer,is_event
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,258611670061820981248,1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,231646719505939324929,1,0,0,0,0,1,0,1,0,0,0,0,0


In [182]:
merged = pd.concat([all_trains, test], axis=1)

In [188]:
merged.timestamp[100].weekday()

1

In [191]:
def get_days(r):
    #Monday is 0 and Sunday is 6.
        return pd.Series([
            1 if r['timestamp'].weekday() == 0 else 0, #monday
            1 if r['timestamp'].weekday() == 1 else 0, 
            1 if r['timestamp'].weekday() == 2 else 0,
            1 if r['timestamp'].weekday() == 3 else 0,
            1 if r['timestamp'].weekday() == 4 else 0,
            1 if r['timestamp'].weekday() == 5 else 0,
            1 if r['timestamp'].weekday() == 6 else 0 #sunday
    ])

In [192]:
days = all_trains.apply(lambda x:get_days(x),axis=1)
days.columns = ['d_monday','d_tuesday','d_wednesday','d_thursday','d_friday','d_saturday','d_sunday',]

In [199]:
merged = pd.concat([merged, days], axis=1)

In [200]:
filename = "./data/merged_concat_final.csv"
merged.to_csv(filename, sep='\t', encoding='utf-8')
print filename

./data/merged_concat_final.csv
